# Set up

In [3]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/My Drive/Team 3 Stats 170A/

/content/drive/My Drive/Team 3 Stats 170A


# Load Data

In [210]:
all_posts = pd.read_csv('subreddit_data/posts/all_posts_2010_2023.csv')

KeyboardInterrupt: ignored

In [ ]:
school = 'ucla'

In [ ]:
sent_comments = pd.read_csv(f'subreddit_data/sentiment_comments/{school}_sentiment_comments.csv')

In [ ]:
list(all_posts.columns.values)

In [ ]:
#list(sent_comments.columns.values)
sent_comments = sent_comments[['id', 'link_id', 'subreddit', 'sentiment_score']].copy()

In [ ]:
# remove t3_ from link_id
sent_comments['link_id'] = sent_comments['link_id'].str[3:]
sent_comments['link_id'][0:10]

# Link Comments to Posts and Get Sentiment of Posts

In [ ]:
df_merged = pd.merge(all_posts, sent_comments, how = 'outer', left_on = 'id', right_on = 'link_id')
df_merged = df_merged.dropna(subset=['sentiment_score', 'id_x'])

In [ ]:
df_merged.head(10)

In [ ]:
df_merged['id_x'][:-100]

In [ ]:
sent_post = df_merged.groupby(['id_x'])['sentiment_score'].mean()

In [ ]:
sent_post[0:10]

In [ ]:
df_post_sent = pd.merge(df_merged, sent_post, how = 'outer', left_on = 'id_x', right_on = sent_post.index)

In [ ]:
df_post_sent.head(10)

In [ ]:
len(df_post_sent)

In [ ]:
df_post_sent = df_post_sent.rename(columns = {'id_x': 'post_id', 'subreddit_x': 'post_subreddit', 'id_y': 'comment_id',
                                              'subreddit_y': 'comment_subreddit', 'sentiment_score_x': 'comment_sentiment_score',
                                              'sentiment_score_y': 'post_sentiment_score'})

In [ ]:
df_post_sent.head(10)

In [ ]:
df_post_sent.to_csv(f'./subreddit_data/sentiment_posts/{school}_post_sentiment_w_comments.csv')

# Sentiment scores for posts without comments

In [24]:
school = 'ucsc'

In [25]:
posts_w_comments = pd.read_csv(f'./subreddit_data/sentiment_posts/{school}_post_sentiment_w_comments.csv')

In [26]:
num_comments = posts_w_comments.groupby(['post_id']).size().reset_index(name = 'num_comments')
num_comments

,post_id,num_comments
0,1002407,2
1,1002lp2,2
2,10046fo,1
3,1006uem,1
4,1007rw9,58
...,...,...
54292,zz7pl,13
54293,zzin7k,9
54294,zzoc0,4
54295,zzprh,7


In [27]:
only_posts = posts_w_comments[['post_id', 'post_subreddit', 'title', 'selftext', 'text', 'created_utc', 'post_sentiment_score']].copy().drop_duplicates()
only_posts.head(10)

,post_id,post_subreddit,title,selftext,text,created_utc,post_sentiment_score
0,gvhpx,UCSC,"Hello, r/UCSC I'm planning on attending in the...",If you guys could give me some info/insight th...,"Hello, r/UCSC I'm planning on attending in the...",2011-04-23 03:01:51,0.342560
25,gjspo,UCSC,Hoping to move into an apartment next year. An...,I'm currently a freshman living in a small tri...,Hoping to move into an apartment next year. An...,2011-04-06 09:33:09,0.327100
43,forlz,UCSC,First Rain...,-___________-,First Rain...-___________-,2011-02-20 01:19:13,0.152281
59,fj3kk,UCSC,Anyone catch the CHP Valentine's Issue?,The artwork section has a familiar face...,Anyone catch the CHP Valentine's Issue?The art...,2011-02-10 23:40:08,-0.038250
61,fdqi5,UCSC,Who is Wesley Mackey?,I've been trying to find some kind of backgrou...,Who is Wesley Mackey?I've been trying to find ...,2011-02-02 11:08:33,0.000000
62,dzu7u,UCSC,"Check out Charlie McDowell, head of the Comput...",NaN,"Check out Charlie McDowell, head of the Comput...",2010-11-02 02:57:29,0.301257
69,yvedx,UCSC,A quick question about textbooks,Hey everyone! I'm going to be a new student at...,A quick question about textbooksHey everyone! ...,2012-08-26 22:14:22,0.515627
80,yv55g,UCSC,Anyone ride mountain bikes?,I just graduated from UCSB in June and I'm mov...,Anyone ride mountain bikes?I just graduated fr...,2012-08-26 19:32:27,0.142300
82,yuvuf,UCSC,America's Coolest (sustainable) schools from S...,NaN,America's Coolest (sustainable) schools from S...,2012-08-26 16:40:50,0.132048
105,ytlj2,UCSC,UCSC parties tonight,Hello! We're three British students staying in...,UCSC parties tonightHello! We're three British...,2012-08-25 20:54:33,0.351530


In [28]:
# posts with comment counts
posts_w_counts = pd.merge(only_posts, num_comments, how = 'outer', left_on = 'post_id', right_on = 'post_id')
posts_w_counts.head(10)

,post_id,post_subreddit,title,selftext,text,created_utc,post_sentiment_score,num_comments
0,gvhpx,UCSC,"Hello, r/UCSC I'm planning on attending in the...",If you guys could give me some info/insight th...,"Hello, r/UCSC I'm planning on attending in the...",2011-04-23 03:01:51,0.342560,25
1,gjspo,UCSC,Hoping to move into an apartment next year. An...,I'm currently a freshman living in a small tri...,Hoping to move into an apartment next year. An...,2011-04-06 09:33:09,0.327100,18
2,forlz,UCSC,First Rain...,-___________-,First Rain...-___________-,2011-02-20 01:19:13,0.152281,16
3,fj3kk,UCSC,Anyone catch the CHP Valentine's Issue?,The artwork section has a familiar face...,Anyone catch the CHP Valentine's Issue?The art...,2011-02-10 23:40:08,-0.038250,2
4,fdqi5,UCSC,Who is Wesley Mackey?,I've been trying to find some kind of backgrou...,Who is Wesley Mackey?I've been trying to find ...,2011-02-02 11:08:33,0.000000,1
5,dzu7u,UCSC,"Check out Charlie McDowell, head of the Comput...",NaN,"Check out Charlie McDowell, head of the Comput...",2010-11-02 02:57:29,0.301257,7
6,yvedx,UCSC,A quick question about textbooks,Hey everyone! I'm going to be a new student at...,A quick question about textbooksHey everyone! ...,2012-08-26 22:14:22,0.515627,11
7,yv55g,UCSC,Anyone ride mountain bikes?,I just graduated from UCSB in June and I'm mov...,Anyone ride mountain bikes?I just graduated fr...,2012-08-26 19:32:27,0.142300,2
8,yuvuf,UCSC,America's Coolest (sustainable) schools from S...,NaN,America's Coolest (sustainable) schools from S...,2012-08-26 16:40:50,0.132048,23
9,ytlj2,UCSC,UCSC parties tonight,Hello! We're three British students staying in...,UCSC parties tonightHello! We're three British...,2012-08-25 20:54:33,0.351530,23


In [29]:
posts_w_counts.to_csv(f'./subreddit_data/sentiment_posts/{school}_post_sentiment.csv')